In [1]:
import mne
import sys
import os
import importlib


from braindecode.preprocessing.preprocess import preprocess, Preprocessor, zscore
from braindecode.datasets import (create_from_mne_raw, create_from_mne_epochs)
from braindecode.preprocessing.windowers import create_windows_from_events
from braindecode.datasets.sleep_physionet import SleepPhysionet
from braindecode.datasets import BaseDataset, BaseConcatDataset, WindowsDataset

from plot import Plot

In [1]:
import mat73
from scipy.io import loadmat

In [3]:
import mne
from mne_extras import write_edf

import os
from tqdm import tqdm

In [60]:

import pprint

pp = pprint.PrettyPrinter(indent=2)

# data_dir = '/media/maligan/My Passport/msc_thesis/ssl_thesis/data/'
data_dir = '/media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/'


In [63]:
files = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir)]
# subject_files = [os.path.join(files[0], fname) for fname in os.listdir(files[0])]
# recordings = [os.path.join(subjects[0], fname) for fname in os.listdir(subjects[0])]

In [64]:
files

['/media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal',
 '/media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/normal']

In [52]:
def get_file_list(x):
    return [os.path.join(x, fname) for fname in os.listdir(x)]

def get_id(x):
    return x.split('/')[-1]

In [208]:
annotations = {}
for annotation in get_file_list(data_dir):
    subjects = {}
    for subject in get_file_list(annotation):
        recordings = {}
        for recording in get_file_list(subject):
            dates = {}
            for date in get_file_list(recording):
                for raw_path in get_file_list(date):
                    if '.edf' in get_id(raw_path):
                        break
                    else:
                        pass
                dates[get_id(date)] = raw_path
            recordings[get_id(recording)] = dates
        subjects[get_id(subject)] = recordings
    annotations[get_id(annotation)] = subjects



pp.pprint(annotations)


{ 'abnormal': { '007': { '00000768': { 's003_2012_04_06': '/media/maligan/My '
                                                          'Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/007/00000768/s003_2012_04_06/00000768_s003_t000.edf'}},
                '011': { '00001154': { 's007_2012_07_25': '/media/maligan/My '
                                                          'Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/011/00001154/s007_2012_07_25/00001154_s007_t000.edf'}},
                '012': { '00001217': { 's002_2012_09_17': '/media/maligan/My '
                                                          'Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/012/00001217/s002_2012_09_17/00001217_s002_t000.edf'}},
                '021': { '00002118': { 's006_2013_08_16': '/media/maligan/My '
                                                          'Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/021/0000

In [434]:
import pandas as pd

df = pd.json_normalize(annotations, sep='_').T

# paths list
raw_paths = [df.iloc[i][0] for i in range(len(df))]

# define abnormal and normal subjects
abnormal_subjects = annotations['abnormal'].keys()
normal_subjects = annotations['normal'].keys()

# define descriptions (recoding per subject)
abnormal_descriptions, normal_descriptions, classification = [], [], []
for id in abnormal_subjects:
    for recording in annotations['abnormal'][id].values():
        for x in recording.keys():
            abnormal_descriptions += [{'subject': id, 'recording': x}]
            classification += ['abnormal']
for id in normal_subjects:
    for recording in annotations['normal'][id].values():
        for x in recording.keys():
            normal_descriptions += [{'subject': id, 'recording': x}]
            classification += ['normal']

descriptions = abnormal_descriptions + normal_descriptions

In [435]:
from sklearn.utils import shuffle
raw_paths, descriptions, classification = shuffle(raw_paths, descriptions, classification)

In [437]:
# [print(id) for id in abnormal_subjects ]
# [print(x) for x in annotations['abnormal']['072']]

# len(descriptions)
# len(raw_paths)
# len(classification)


In [357]:

raw = mne.io.read_raw_edf('/media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/007/00000768/s003_2012_04_06/00000768_s003_t000.edf')

Extracting EDF parameters from /media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/007/00000768/s003_2012_04_06/00000768_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [387]:
sfreq = 100
high_cut_hz = 30
low_cut_hz = 0.5
n_jobs = 1

# preprocessing
# for raw in raws:
mne.io.Raw.resample(raw, sfreq)   # resample
mne.io.Raw.filter(raw, l_freq=low_cut_hz, h_freq=high_cut_hz, n_jobs=n_jobs)    # high-pass filter

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 661 samples (6.610 sec)



<RawEDF | 00000768_s003_t000.edf, 30 x 121700 (1217.0 s), ~27.9 MB, data loaded>

In [402]:
raw = raw.set_annotations(mne.Annotations(onset=[0], duration=raw.times.max(), description=['abnormal']))

In [331]:
ssl_sample = mne.io.read_raw_edf('/media/maligan/My Passport/msc_thesis/ssl_thesis/data/sleep_staging_sample/SC4012E0-PSG.edf')
ssl_sample

Extracting EDF parameters from /media/maligan/My Passport/msc_thesis/ssl_thesis/data/sleep_staging_sample/SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<Annotations | 0 segments>

,0
abnormal_072_00007221_s001_2010_10_27,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_072_00007221_s002_2010_11_01,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_007_00000768_s003_2012_04_06,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_011_00001154_s007_2012_07_25,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_012_00001217_s002_2012_09_17,/media/maligan/My Passport/msc_thesis/ssl_thes...
...,...
normal_105_00010577_s001_2013_07_24,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_106_00010660_s001_2013_08_19,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_106_00010680_s001_2013_08_22,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_107_00010725_s001_2013_09_10,/media/maligan/My Passport/msc_thesis/ssl_thes...


In [428]:

# load data and set annotations
dataset = []
for i, path in enumerate(raw_paths):
    # classification = 'abnormal' if  else 'normal'
    print(df.iloc[i].name.split('_')[0])
    
    # raw = mne.io.read_raw_fif(path)
    # raw = raw.set_annotations(mne.Annotations(onset=[0], duration=raw.times.max(), description=['abnormal']))
    # dataset.append(raw)


abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
abnormal
a

In [153]:
raw = annotations['abnormal']['021']['00002118']['s006_2013_08_16']
raw = mne.io.read_raw_edf(raw)
# raw.ch_names

Extracting EDF parameters from /media/maligan/My Passport/msc_thesis/ssl_thesis/data/tuh_abnormal_data/eval/abnormal/021/00002118/s006_2013_08_16/00002118_s006_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [142]:
import pandas as pd

df = pd.json_normalize(annotations, sep='_').T
df

,0
abnormal_072_00007221_s001_2010_10_27,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_072_00007221_s002_2010_11_01,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_007_00000768_s003_2012_04_06,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_011_00001154_s007_2012_07_25,/media/maligan/My Passport/msc_thesis/ssl_thes...
abnormal_012_00001217_s002_2012_09_17,/media/maligan/My Passport/msc_thesis/ssl_thes...
...,...
normal_105_00010577_s001_2013_07_24,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_106_00010660_s001_2013_08_19,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_106_00010680_s001_2013_08_22,/media/maligan/My Passport/msc_thesis/ssl_thes...
normal_107_00010725_s001_2013_09_10,/media/maligan/My Passport/msc_thesis/ssl_thes...


In [ ]:
# read, pick channels, save loop
for i in tqdm(range(len(df))):
    file = df.iloc[i][0]
    raw = mne.io.read_raw_edf(file)
    to_export = f"{file.split('/')[-1].split('.')[0]}"
    raw.save(f"{file}_2_channels.fif", picks=['EEG FZ-REF', 'EEG PZ-REF'], overwrite=True)


In [194]:
# shuffle two arrays the same way

import random

desc

random.seed(90)
random.shuffle(descriptions)
# reset the same seed to get the identical random sequence and shuffle the y
# random.seed(seed)
random.shuffle(y_data)

NameError: name 'desc' is not defined

In [255]:
pp.pprint(len(normal_descriptions+abnormal_descriptions))
pp.pprint(len(raw_paths))

247
269


In [ ]:
# read, pick channels, save loop
for file in tqdm(files):
    raw = mne.io.read_raw_edf('{file}')
    to_export = f"{file.split('/')[-1]}"
    raw.save(f"{data_dir}../eval_2_channels/{to_export}", picks=['Fpz', 'Pz'], overwrite=True)


In [22]:
data = mat73.loadmat('data/scopolamine/mats/CHDR0511.S2044.10MAY06_0942.O01M01.mat')
biomRegName = loadmat('data/scopolamine/biomRegName.mat')
classification = loadmat('data/scopolamine/ClassificationStatObj.mat')
dataMatrix = loadmat('data/scopolamine/DataMatrixScp1Scp5.mat')
mAChRindex = loadmat('data/scopolamine/mAChRindex.mat')
outcome = loadmat('data/scopolamine/Outcome.mat')

In [31]:
data['RawSignal']

array([[ 0.49962461, -0.65875173, -0.14366271,  0.01985283, -0.04261129,
        -0.02809451],
       [ 0.49962461, -0.65239894, -0.14367796,  0.01198544, -0.04263961,
        -0.02780316],
       [ 0.49962461, -0.66252047, -0.14350288,  0.02470998, -0.04259126,
        -0.0282714 ],
       ...,
       [ 0.49962461, -0.13753186,  0.04217241,  0.28275687, -0.04272525,
        -0.03777108],
       [ 0.49962461, -0.12997124,  0.0539127 ,  0.27765587, -0.04302568,
        -0.03681615],
       [ 0.49962461, -0.13421607,  0.0474169 ,  0.28051439, -0.04286418,
        -0.03732444]])

In [34]:
mne.io.RawArray(data['RawSignal'], mne.create_info(['Fz-Cz', 'Pz-Oz', '3','4','5','6'], 100))

ValueError: len(data) (64768) does not match len(info["ch_names"]) (6)

In [155]:

print('LOADING BCI DATA')

''' ANNOTATIONS
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
    the left fist (in runs 3, 4, 7, 8, 11, and 12)
    both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
    the right fist (in runs 3, 4, 7, 8, 11, and 12)
    both feet (in runs 5, 6, 9, 10, 13, and 14)
'''

subject_size = [1,10]
subjects = range(subject_size[0], subject_size[1]) # max 110
event_codes = [
    1, 2, # eyes open, eyes closed (baselines)
    3, 4, 5,
    6, 7, 8, 9, 
    10, 11, 12, 13, 14
]

physionet_paths, descriptions = [], []

for subject_id in subjects:
    physionet_paths += [mne.datasets.eegbci.load_data(subject_id, event_codes, update_path=False)]
    descriptions += [{"event_code": code, "subject": subject_id} for code in event_codes]


LOADING BCI DATA


In [197]:
# raw_set = [
#     '/home/maligan/mne_data/physionet-sleep-data/SC4012E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4451F0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4441E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4431E0-PSG.edf',
#     '/home/maligan/mne_data/physionet-sleep-data/SC4421E0-PSG.edf',
# ]

# space_bambi directory
data_dir = './data/SPACE_BAMBI_2channels/'
raw_set = [
    f'{data_dir}SPACE.S175.yyyymmdd.EORASD_raw.fif',
    f'{data_dir}SPACE.S170.yyyymmdd.EORASD_raw.fif'
]

print(raw_set)

# load into raw array
raws = [mne.io.read_raw_fif(x, preload=True) for x in raw_set]

['./data/SPACE_BAMBI_2channels/SPACE.S175.yyyymmdd.EORASD_raw.fif', './data/SPACE_BAMBI_2channels/SPACE.S170.yyyymmdd.EORASD_raw.fif']
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S175.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 628735 =      0.000 ...   307.000 secs
Ready.
Reading 0 ... 628735  =      0.000 ...   307.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S170.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 616447 =      0.000 ...   301.000 secs
Ready.
Reading 0 ... 616447  =      0.000 ...   301.000 secs...


In [3]:
# space_bambi directory
data_dir = './data/SPACE_BAMBI_2channels/'

raws = []

print(f'{len(os.listdir(data_dir))} files found')
for i, path in enumerate(os.listdir(data_dir)):
    if i == 5:
        break
    full_path = os.path.join(data_dir, path)
    raws.append(mne.io.read_raw_fif(full_path, preload=True))

340 files found
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S27.yyyymmdd.ECRASD_raw.fif...
    Range : 0 ... 368639 =      0.000 ...   180.000 secs
Ready.
Reading 0 ... 368639  =      0.000 ...   180.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/BAMBI.S620.yyyymmdd.EORASD1_raw.fif...
    Range : 0 ... 618495 =      0.000 ...   302.000 secs
Ready.
Reading 0 ... 618495  =      0.000 ...   302.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S50.yyyymmdd.EORASD_raw.fif...
    Range : 0 ... 378879 =      0.000 ...   185.000 secs
Ready.
Reading 0 ... 378879  =      0.000 ...   185.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S101.yyyymmdd.EORHE_raw.fif...
    Range : 0 ... 622591 =      0.000 ...   304.000 secs
Ready.
Reading 0 ... 622591  =      0.000 ...   304.000 secs...
Opening raw data file ./data/SPACE_BAMBI_2channels/SPACE.S94.yyyymmdd.EORHE_raw.fif...
    Range : 0 ... 731135 =      0.000 ...   357.000 secs
Re

In [164]:
# import pandas as pd
# descriptions = []

# for id, raw in enumerate(raws):
#     descriptions += [{
#         id: {
#         "onset": annot['onset'], 
#         "duration": annot['duration'], 
#         "description": annot['description'], 
#         "orig_time": annot['orig_time']
#         }
#     } for annot in raw.annotations]

In [199]:
descriptions = []

for subject_id, raw in enumerate(raws):
    descriptions += [{"subject": subject_id}]

In [202]:
descriptions[0]

{'event_code': 1, 'subject': 1}

In [205]:
raws[0].ch_names

['Fpz', 'Pz']

In [201]:

event_codes = [
    1, 2, # eyes open, eyes closed (baselines)
    3, 4, 5,
    6, 7, 8, 9, 
    10, 11, 12, 13, 14
]

physionet_paths, descriptions = [], []

for subject_id in subjects:
    physionet_paths += [mne.datasets.eegbci.load_data(subject_id, event_codes, update_path=False)]
    descriptions += [{"event_code": code, "subject": subject_id} for code in event_codes]


In [76]:
descriptions

[{'event_code': 1, 'subject': 1},
 {'event_code': 2, 'subject': 1},
 {'event_code': 3, 'subject': 1},
 {'event_code': 4, 'subject': 1},
 {'event_code': 5, 'subject': 1},
 {'event_code': 6, 'subject': 1},
 {'event_code': 7, 'subject': 1},
 {'event_code': 8, 'subject': 1},
 {'event_code': 9, 'subject': 1},
 {'event_code': 10, 'subject': 1},
 {'event_code': 11, 'subject': 1},
 {'event_code': 12, 'subject': 1},
 {'event_code': 13, 'subject': 1},
 {'event_code': 14, 'subject': 1},
 {'event_code': 1, 'subject': 2},
 {'event_code': 2, 'subject': 2},
 {'event_code': 3, 'subject': 2},
 {'event_code': 4, 'subject': 2},
 {'event_code': 5, 'subject': 2},
 {'event_code': 6, 'subject': 2},
 {'event_code': 7, 'subject': 2},
 {'event_code': 8, 'subject': 2},
 {'event_code': 9, 'subject': 2},
 {'event_code': 10, 'subject': 2},
 {'event_code': 11, 'subject': 2},
 {'event_code': 12, 'subject': 2},
 {'event_code': 13, 'subject': 2},
 {'event_code': 14, 'subject': 2},
 {'event_code': 1, 'subject': 3},
 {'e

In [4]:
### preprocess
# resample to 160Hz
# high pass filtering of 30Hz

sfreq = 160
high_cut_hz = 30
n_jobs = 2

for raw in raws:
    mne.io.Raw.resample(raw, sfreq, n_jobs=n_jobs) # resample all files
    mne.io.Raw.filter(raw, l_freq=None, h_freq=high_cut_hz, n_jobs=n_jobs) # high-pass filter

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 71 samples (0.444 sec)



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished


In [5]:
base_datasets = [BaseDataset(raw) for raw in raws]
base_concat_datasets = BaseConcatDataset(base_datasets)

In [6]:
base_concat_datasets

In [7]:
window_size_samples = 100

windows_dataset = create_from_mne_raw(
    raws,
    trial_start_offset_samples=0,
    trial_stop_offset_samples=0,
    window_size_samples=window_size_samples,
    window_stride_samples=window_size_samples,
    drop_last_window=True,
    # descriptions=descriptions,
    accepted_bads_ratio=0.5,
    drop_bad_windows=True,
    # mapping=mapping,
    # preload=True,
    on_missing='ignore'
)


Used Annotations descriptions: ['artifact']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
112 matching events found
No baseline correction applied
0 projection items activated
Loading data for 112 events and 100 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['artifact']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 100 original time points ...
0 bad epochs dropped


In [8]:
type(windows_dataset)

braindecode.datasets.base.BaseConcatDataset

In [9]:
# z-score normalization

preprocess(windows_dataset, Preprocessor(zscore))


Loading data for 112 events and 100 original time points ...
Loading data for 120 events and 100 original time points ...


/home/maligan/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function zscore is deprecated; will be removed in 0.7.0. Use sklearn.preprocessing.scale instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: 'WindowsDataset' object has no attribute '__annotations__'